# TITANIC

Features originales y features random para ver el impacto de la cardinalidad de numericas y categoricas

https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-py


In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from networkx.drawing.nx_pydot import write_dot
from networkx.linalg.graphmatrix import *
from networkx.algorithms.approximation import *
#from networkx.centrality import closn
from networkx.algorithms import *
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import OneHotEncoder
import gc
gc.collect()

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV

In [2]:
import sys
import os
module_path = os.path.abspath('C:\\Users\\Matteo\\Documents\\work in progress\\GRAPH_NN\\XaiGraph')
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from datasets.titanic_cooking import titanic_cooking

X_train_prepro, X_test_prepro = titanic_cooking()

TRAIN
    age  count
0  0.85    128
1  0.15    119
2  0.55    117
3  0.45    109
4  0.75    106
5  0.25    102
6  0.35    101
7  0.05     96
8  0.65     90
9  0.95     79
TEST
    age  count
0  0.85     36
1  0.55     30
2  0.35     28
3  0.75     28
4  0.25     26
5  0.65     24
6  0.95     24
7  0.05     23
8  0.45     23
9  0.15     20
TRAIN
   fare  count
0  0.15    199
1  0.65    145
2  0.85    145
3  0.25    123
4  0.35     97
5  0.45     89
6  0.75     86
7  0.55     83
8  0.95     55
9  0.05     25
TEST
   fare  count
0  0.65     47
1  0.15     44
2  0.85     43
3  0.45     24
4  0.25     23
5  0.55     23
6  0.75     21
7  0.95     16
8  0.35     13
9  0.05      8
TRAIN
   family_size  count
0          0.0    643
1          1.0    178
2          2.0    123
3          3.0     35
4          4.0     20
5          5.0     20
6          6.0     15
7         10.0      7
8          7.0      6
TEST
   family_size  count
0          0.0    147
1          1.0     57
2          2.0     36

C:\Users\Matteo\matteo_virtualenv\graph_nn\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Matteo\Documents\work in progress\GRAPH_NN\XaiGraph\datasets\titanic_cooking.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['family_size'] = dataset['parch'] + dataset['sibsp']
C:\Users\Matteo\Documents\work in progress\GRAPH_NN\XaiGraph\datasets\titanic_cooking.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [4]:
X_train_prepro

,age,fare,family_size,embarked,sex,pclass,title,is_alone,target
0,0.75,0.15,0.0,Q,female,3.0,Mrs,1,1.0
1,0.35,0.65,1.0,C,female,2.0,Mrs,1,1.0
2,0.05,0.75,6.0,S,female,3.0,Mrs,0,0.0
3,0.35,0.15,0.0,C,male,3.0,Mr,1,0.0
4,0.15,0.15,0.0,S,female,3.0,Mrs,1,1.0
...,...,...,...,...,...,...,...,...,...
1042,0.85,0.15,1.0,S,female,3.0,Mrs,1,1.0
1043,0.05,0.75,5.0,S,male,3.0,Master,0,0.0
1044,0.15,0.25,2.0,S,male,3.0,Mr,0,0.0
1045,0.15,0.35,0.0,S,male,2.0,Mr,1,0.0


In [5]:
cols = list(X_train_prepro.columns)[:-1] + ['rcat', 'rnum', 'target']
rng = np.random.RandomState(seed=42)
X_train_prepro["rcat"] = rng.randint(3, size=X_train_prepro.shape[0]).astype(dtype=np.float32)
X_train_prepro["rnum"] = rng.randint(50, size=X_train_prepro.shape[0]).astype(dtype=np.float32)

X_train_prepro = X_train_prepro[cols]
X_train_prepro

,age,fare,family_size,embarked,sex,pclass,title,is_alone,rcat,rnum,target
0,0.75,0.15,0.0,Q,female,3.0,Mrs,1,2.0,4.0,1.0
1,0.35,0.65,1.0,C,female,2.0,Mrs,1,0.0,22.0,1.0
2,0.05,0.75,6.0,S,female,3.0,Mrs,0,2.0,9.0,0.0
3,0.35,0.15,0.0,C,male,3.0,Mr,1,2.0,43.0,0.0
4,0.15,0.15,0.0,S,female,3.0,Mrs,1,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1042,0.85,0.15,1.0,S,female,3.0,Mrs,1,2.0,13.0,1.0
1043,0.05,0.75,5.0,S,male,3.0,Master,0,0.0,13.0,0.0
1044,0.15,0.25,2.0,S,male,3.0,Mr,0,1.0,1.0,0.0
1045,0.15,0.35,0.0,S,male,2.0,Mr,1,1.0,36.0,0.0


In [6]:
X_test_prepro["rcat"] = rng.randint(3, size=X_test_prepro.shape[0]).astype(dtype=np.float32)
X_test_prepro["rnum"] = rng.randint(50, size=X_test_prepro.shape[0]).astype(dtype=np.float32)

X_test_prepro = X_test_prepro[cols]
X_test_prepro

,age,fare,family_size,embarked,sex,pclass,title,is_alone,rcat,rnum,target
0,0.55,0.65,1.0,Q,female,3.0,Mrs,1,1.0,41.0,1.0
1,0.15,0.65,2.0,C,male,3.0,Master,0,0.0,35.0,1.0
2,0.65,0.15,0.0,S,male,3.0,Mr,1,0.0,19.0,0.0
3,0.55,0.25,0.0,S,male,3.0,Mr,1,1.0,3.0,0.0
4,0.65,0.15,0.0,S,male,3.0,Mr,1,1.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
257,0.25,0.15,0.0,S,male,3.0,Mr,1,2.0,44.0,0.0
258,0.55,0.55,1.0,S,female,3.0,Mrs,1,2.0,11.0,0.0
259,0.85,0.05,0.0,S,male,1.0,Mr,1,2.0,31.0,0.0
260,0.15,0.35,0.0,S,male,3.0,Mr,1,0.0,31.0,0.0


In [7]:
# Train-Test a model 
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder

categorical_columns = ["family_size", "embarked", "sex", "pclass", "title", "is_alone", "rcat"]
#categorical_columns = ["family_size", "embarked", "sex", "pclass", "title", "rcat"]
categorical_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

preprocessing = ColumnTransformer([("cat", categorical_encoder, categorical_columns)])

model = Pipeline(
    [
        ("preprocess", preprocessing),
        ("classifier", RandomForestClassifier(random_state=42)),
    ]
)

model_df = X_train_prepro
#model_df = X_train_prepro.drop(['is_alone'], axis=1) 
print(model_df.columns)
model.fit(model_df.drop(['target'], axis=1), model_df['target'])
train_predict_acc = model.predict(model_df.drop(['target'], axis=1))
test_predict_acc = model.predict(X_test_prepro.drop(['target'], axis=1))
print('TRAIN ACCURACY: {:.4f}'.format(accuracy_score(model_df['target'].values, train_predict_acc)))
print('TEST ACCURACY: {:.4f}'.format(accuracy_score(X_test_prepro['target'].values, test_predict_acc)))

Index(['age', 'fare', 'family_size', 'embarked', 'sex', 'pclass', 'title',
       'is_alone', 'rcat', 'rnum', 'target'],
      dtype='object')
TRAIN ACCURACY: 0.8424
TEST ACCURACY: 0.8244


In [18]:
tmp = X_train_prepro.drop(['target'], axis=1)
tmp['target'] = train_predict_acc

tmp_test = X_test_prepro.drop(['target'], axis=1)
tmp_test['target'] = test_predict_acc

In [9]:
tmp

,age,fare,family_size,embarked,sex,pclass,title,is_alone,rcat,rnum,target
0,0.75,0.15,0.0,Q,female,3.0,Mrs,1,2.0,4.0,0.0
1,0.35,0.65,1.0,C,female,2.0,Mrs,1,0.0,22.0,1.0
2,0.05,0.75,6.0,S,female,3.0,Mrs,0,2.0,9.0,0.0
3,0.35,0.15,0.0,C,male,3.0,Mr,1,2.0,43.0,0.0
4,0.15,0.15,0.0,S,female,3.0,Mrs,1,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1042,0.85,0.15,1.0,S,female,3.0,Mrs,1,2.0,13.0,0.0
1043,0.05,0.75,5.0,S,male,3.0,Master,0,0.0,13.0,0.0
1044,0.15,0.25,2.0,S,male,3.0,Mr,0,1.0,1.0,0.0
1045,0.15,0.35,0.0,S,male,2.0,Mr,1,1.0,36.0,0.0


In [19]:
tmp_test

,age,fare,family_size,embarked,sex,pclass,title,is_alone,rcat,rnum,target
0,0.55,0.65,1.0,Q,female,3.0,Mrs,1,1.0,41.0,1.0
1,0.15,0.65,2.0,C,male,3.0,Master,0,0.0,35.0,0.0
2,0.65,0.15,0.0,S,male,3.0,Mr,1,0.0,19.0,0.0
3,0.55,0.25,0.0,S,male,3.0,Mr,1,1.0,3.0,0.0
4,0.65,0.15,0.0,S,male,3.0,Mr,1,1.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
257,0.25,0.15,0.0,S,male,3.0,Mr,1,2.0,44.0,0.0
258,0.55,0.55,1.0,S,female,3.0,Mrs,1,2.0,11.0,0.0
259,0.85,0.05,0.0,S,male,1.0,Mr,1,2.0,31.0,0.0
260,0.15,0.35,0.0,S,male,3.0,Mr,1,0.0,31.0,0.0


# EXPLICACIÓN GLOBAL

In [215]:
from scipy.special import rel_entr
def myjensenshannon(p, q, base=None, *, axis=0, keepdims=False, median_bool=True):
    """
    Compute the Jensen-Shannon distance (metric) between
    two probability arrays. This is the square root
    of the Jensen-Shannon divergence.
    The Jensen-Shannon distance between two probability
    vectors `p` and `q` is defined as,
    .. math::
       \\sqrt{\\frac{D(p \\parallel m) + D(q \\parallel m)}{2}}
    where :math:`m` is the pointwise mean of :math:`p` and :math:`q`
    and :math:`D` is the Kullback-Leibler divergence.
    This routine will normalize `p` and `q` if they don't sum to 1.0.
    Parameters
    ----------
    p : (N,) array_like
        left probability vector
    q : (N,) array_like
        right probability vector
    base : double, optional
        the base of the logarithm used to compute the output
        if not given, then the routine uses the default base of
        scipy.stats.entropy.
    axis : int, optional
        Axis along which the Jensen-Shannon distances are computed. The default
        is 0.
        .. versionadded:: 1.7.0
    keepdims : bool, optional
        If this is set to `True`, the reduced axes are left in the
        result as dimensions with size one. With this option,
        the result will broadcast correctly against the input array.
        Default is False.
        .. versionadded:: 1.7.0
    Returns
    -------
    js : double or ndarray
        The Jensen-Shannon distances between `p` and `q` along the `axis`.
    Notes
    -----
    .. versionadded:: 1.2.0
    Examples
    --------
    >>> from scipy.spatial import distance
    >>> distance.jensenshannon([1.0, 0.0, 0.0], [0.0, 1.0, 0.0], 2.0)
    1.0
    >>> distance.jensenshannon([1.0, 0.0], [0.5, 0.5])
    0.46450140402245893
    >>> distance.jensenshannon([1.0, 0.0, 0.0], [1.0, 0.0, 0.0])
    0.0
    >>> a = np.array([[1, 2, 3, 4],
    ...               [5, 6, 7, 8],
    ...               [9, 10, 11, 12]])
    >>> b = np.array([[13, 14, 15, 16],
    ...               [17, 18, 19, 20],
    ...               [21, 22, 23, 24]])
    >>> distance.jensenshannon(a, b, axis=0)
    array([0.1954288, 0.1447697, 0.1138377, 0.0927636])
    >>> distance.jensenshannon(a, b, axis=1)
    array([0.1402339, 0.0399106, 0.0201815])
    """
    p = np.asarray(p)
    q = np.asarray(q)
    p = p / np.sum(p, axis=axis, keepdims=True)
    q = q / np.sum(q, axis=axis, keepdims=True)
    m = (p + q) / 2.0
    left = rel_entr(p, m)
    right = rel_entr(q, m)
    #left_sum = np.sum(left, axis=axis, keepdims=keepdims)
    #right_sum = np.sum(right, axis=axis, keepdims=keepdims)
    #js = left_sum + right_sum
    #if base is not None:
    #    js /= np.log(base)
    js_alt = (left + right)
    if median_bool:
        res = np.median(js_alt) / 2.0
    else:
        res = np.sum(js_alt) / 2.0    
    return np.sqrt(res)

## Jensen-Shannon 

Para analisis categórico (targets binarios o multidimensional-->N problemas binarios)

In [216]:
from scipy.spatial import distance

for tipo_test, df_2_ana in [('TRAIN', tmp), ('TEST', tmp_test)]:
    res_var_distr = {}
    for c in df_2_ana.columns: 
        if c == 'target':
            continue

        vals = df_2_ana[c].unique()
        #print('***************************')
        #print(c)
        #print(vals)

        cdf = df_2_ana
        den = len(cdf)
        distr_ref = np.array([len(cdf[cdf[c] == v])/den for v in vals])
        #print(distr_ref)

        cdf = df_2_ana[df_2_ana['target'] > 0]
        den = len(cdf)
        distr_1 = np.array([len(cdf[cdf[c] == v])/den for v in vals])
        #print(distr_1)

        cdf = df_2_ana[df_2_ana['target'] < 1]
        den = len(cdf)
        distr_0 = np.array([len(cdf[cdf[c] == v])/den for v in vals])
        #print(distr_0)

        res_var_distr[c] = myjensenshannon(distr_1, distr_0, median_bool=True) #/ np.sqrt(len(vals))
        show_plot = False
        if show_plot:
            plt.plot(distr_1, label='T1')
            plt.plot(distr_0, label='T0')
            plt.legend()
            plt.title(c + ': {:.3f}'.format(res_var_distr[c]))
            plt.show()

    print('SALIDA {}:'.format(tipo_test))
    for k, v in {k: v for k, v in sorted(res_var_distr.items(), key=lambda item: item[1])}.items():
        print(k, round(100*v, 3))
    print('')

SALIDA TRAIN:
rcat 1.911
rnum 1.96
age 2.415
family_size 5.821
fare 7.641
embarked 8.026
is_alone 12.945
pclass 14.514
title 28.245
sex 39.146

SALIDA TEST:
rnum 3.114
rcat 3.307
age 4.277
family_size 6.348
embarked 7.168
is_alone 8.013
fare 10.977
pclass 17.654
title 26.792
sex 36.268



A falta de validar de una forma más consistente este enfoque:

Potenciales **PROS**:
* Rápido computacionalmente: 2*num_features calculos y escalable (groupby + jensen_shannon de array O(1000) como mucho)
* No sufre el problema de colinearidad/correlación de las variables: dos variables colineares deberían sacar una medida de Jensen-Shannon parecidas
* Normalizando la distancia Jensen-Shannon a la raiz cuadrada del número de valores unicos de la distribución, se deberían hacer comparables las medidas de jensen-shannon de diferentes columnas. 

Potenciales **CONTRA**:
* Se basa en previamente discretizar todas las variables, incluso las continuas, esto en general ya introduce un "error"
* La idea se muy sencilla (¿¿¿¿¿demasiado?????): estamos comparando si la distribución de una feature cambia entre el conjunto etiquetado por el modelo complejo como target 1 y el conjunto etiquetado por el modelo complejo como target 0. Seguro que esto permite sacar insight, pero apostaría que nos estamos perdiendo relaciones entre variables...algún contra-elemplo para ponerlo en evidencia? --> sí cuando haya interacción entre features :(

A continuación calculamos la "permutation_importance" usando scikit-learn para tener un punto de comparación por los resultados obtenidos 

In [74]:
from sklearn.inspection import permutation_importance
r = permutation_importance(model, model_df.drop(['target'], axis=1), model_df['target'],
                            n_repeats=100,
                            random_state=0)

In [75]:
for i in r.importances_mean.argsort()[::-1]:
    #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{list(model_df.columns)[i]:<8}", f"{r.importances_mean[i]:.3f}",
              f" +/- {r.importances_std[i]:.3f}")

sex      0.124  +/- 0.010
pclass   0.071  +/- 0.007
title    0.066  +/- 0.008
family_size 0.040  +/- 0.006
rcat     0.035  +/- 0.006
embarked 0.034  +/- 0.005
is_alone 0.015  +/- 0.004
rnum     0.000  +/- 0.000
fare     0.000  +/- 0.000
age      0.000  +/- 0.000


In [76]:
from sklearn.inspection import permutation_importance
r = permutation_importance(model, tmp_test.drop(['target'], axis=1), tmp_test['target'],
                            n_repeats=100,
                            random_state=0)

In [77]:
for i in r.importances_mean.argsort()[::-1]:
    #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{list(tmp_test.columns)[i]:<8}", f"{r.importances_mean[i]:.3f}",
              f" +/- {r.importances_std[i]:.3f}")

sex      0.217  +/- 0.018
pclass   0.131  +/- 0.017
title    0.123  +/- 0.013
family_size 0.099  +/- 0.012
rcat     0.083  +/- 0.013
embarked 0.070  +/- 0.012
is_alone 0.036  +/- 0.007
rnum     0.000  +/- 0.000
fare     0.000  +/- 0.000
age      0.000  +/- 0.000


NOTA: Como es conocido, "permutation_importance" sufre cuando hay variables colineales. Por ejemplo cuando hacemos el shuffle de la variable is_alone, la prestación del modelo no se ve muy afectada porque el modelo puede recuperar una información equivalente a is_alone desde la variable family_size. Esto justifica que hacer el shuffle de is_alone no parece estropear demasiado las prestaciones. 

## Jensen-Shannon con pares de variables --> IR a la V2...esta no funciona..

In [220]:
from scipy.spatial import distance

cols = [c for c in tmp.columns if c!='target']

for df_2_ana in [tmp, tmp_test]:
    # Calculamos la distribución empiricas por parejas de variables en el caso de target=1 (res_1) y en el caso de target=0 (res_0)
    res_1 = {}
    res_0 = {}
    tmp1 = df_2_ana[df_2_ana['target'] > 0]
    tmp0 = df_2_ana[df_2_ana['target'] < 1]
    den1 = len(tmp1)
    den0 = len(tmp0)
    for i1 in range(len(cols)): 
        c1 = cols[i1]
        for i2 in range(i1+1, len(cols)):  #for i2 in range(i1+1, len(cols)):
            c2 = cols[i2]
            tmp_df = tmp1.groupby([c1, c2]).size().sort_values(ascending=False).reset_index(name='count')
            c1_v = tmp_df[c1].to_list()
            c2_v = tmp_df[c2].to_list()
            counts = tmp_df['count'].to_list()
            res_1[(c1, c2)] = {(c1 + '_{}'.format(v1), c2 + '_{}'.format(v2)): cc for v1, v2, cc in zip(c1_v, c2_v, counts)}
            tmp_df = tmp0.groupby([c1, c2]).size().sort_values(ascending=False).reset_index(name='count')
            c1_v = tmp_df[c1].to_list()
            c2_v = tmp_df[c2].to_list()
            counts = tmp_df['count'].to_list()
            res_0[(c1, c2)] = {(c1 + '_{}'.format(v1), c2 + '_{}'.format(v2)): cc for v1, v2, cc in zip(c1_v, c2_v, counts)}

    cols_values = {}
    for ccc in cols: 
        ccc_v = df_2_ana[ccc].unique().tolist()
        cols_values[ccc] = [ccc + '_{}'.format(v_t) for v_t in ccc_v]
        
    # Para cada pareja de features (f_i, f_j) tenemos la distribución para el subconjunto de target=1 (p_1) y para el subconjunto de target=0
    # (p_0) --> Calculamos la distancia Jensen-Shannon entre p_1 y p_0
    js_res = {}
    for c1 in cols:
        c1_v = cols_values[c1]
        for c2 in cols: 
            if c1 == c2:
                continue
            c2_v = cols_values[c2]
            res0_tmp = res_0.get((c1,c2), None)
            inv = False
            if res0_tmp is None:
                res0_tmp = res_0.get((c2,c1), None)
                res1_tmp = res_1.get((c2,c1), None)
                inv = True
                
            jss = []
            for v1 in c1_v:
                p_0 = []
                p_1 = []
                for v2 in c2_v:
                    p_0.append(res0_tmp.get((v1, v2), 0.0) if not inv else res0_tmp.get((v2, v1), 0.0))
                    p_1.append(res1_tmp.get((v1, v2), 0.0) if not inv else res1_tmp.get((v2, v1), 0.0))
                #print(c2_v)
                #print(p_0)
                #print(p_1)
                if sum(p_0) == 0:
                    p_0 = [1 for _ in range(len(p_0))]
                if sum(p_1) == 0:
                    p_1 = [1 for _ in range(len(p_1))]
                js_tmp = myjensenshannon(p_1, p_0)
                if np.isnan(js_tmp):
                    #print(c1, c2)
                    #print(p_1)
                    #print(p_0)
                    js_tmp = 0.0 #np.sqrt(np.log(2)/2) * np.max(np.array([np.sum(np.array(p_1) > 0), np.sum(np.array(p_0) > 0)]))/len(p_1)
                jss.append(js_tmp)
                #print(v1, jss[-1])
            #print(c1, jss)
            js_res[(c1, c2)] = np.array(jss)
            
    #print(js_res['age', 'rnum'])
    #print(js_res['rnum', 'age'])
           
    res = {}
    for c in cols:
        res_c = []
        for k, v in js_res.items():
            if k[0] == c:
                res_c.append(v.tolist())
                #print(k, c, v)
        #print(res_c)
        res[c] = np.median(np.array(res_c))
        

    print('================================================')
    print('ORIGINAL')
    for k, v in {k: v for k, v in sorted(res.items(), key=lambda item: item[1])}.items():
    #for k, v in res.items():
        print(k, v)


ORIGINAL
is_alone 0.06695509098183382
pclass 0.07266849053245086
embarked 0.0752463101611492
sex 0.08716992084369285
title 0.09670625674741332
rcat 0.1006803991550722
family_size 0.1286008767345432
fare 0.1628333372046611
rnum 0.16827509064792154
age 0.17718535098152408
ORIGINAL
pclass 0.08325546111576977
sex 0.08522848546695433
title 0.0913704166416697
is_alone 0.09695103567765848
rcat 0.11545364235523614
embarked 0.128439863660842
fare 0.1827660568162443
age 0.1861648705529517
family_size 0.1861648705529517
rnum 0.20609405425399666


## Jensen-Shannon por pares v2

In [227]:
from scipy.spatial import distance

cols = [c for c in tmp.columns if c!='target']

for tipo_test, df_2_ana in [('TRAIN', tmp), ('TEST', tmp_test)]:
    # Calculamos la distribución empiricas por parejas de variables en el caso de target=1 (res_1) y en el caso de target=0 (res_0)
    res_1 = {}
    res_0 = {}
    tmp1 = df_2_ana[df_2_ana['target'] > 0]
    tmp0 = df_2_ana[df_2_ana['target'] < 1]
    den1 = len(tmp1)
    den0 = len(tmp0)
    for i1 in range(len(cols)): 
        c1 = cols[i1]
        for i2 in range(i1+1, len(cols)):
            c2 = cols[i2]
            tmp_df = tmp1.groupby([c1, c2]).size().sort_values(ascending=False).reset_index(name='count')
            c1_v = tmp_df[c1].to_list()
            c2_v = tmp_df[c2].to_list()
            counts = tmp_df['count'].to_list()
            #res_1[(c1, c2)] = {(c1 + '_{}'.format(v1), c2 + '_{}'.format(v2)): cc/den1/(len(cols)-1) for v1, v2, cc in zip(c1_v, c2_v, counts)}
            res_1[(c1, c2)] = {(c1 + '_{}'.format(v1), c2 + '_{}'.format(v2)): cc for v1, v2, cc in zip(c1_v, c2_v, counts)}



            tmp_df = tmp0.groupby([c1, c2]).size().sort_values(ascending=False).reset_index(name='count')
            c1_v = tmp_df[c1].to_list()
            c2_v = tmp_df[c2].to_list()
            counts = tmp_df['count'].to_list()
            #res_0[(c1, c2)] = {(c1 + '_{}'.format(v1), c2 + '_{}'.format(v2)): cc/den0/(len(cols)-1) for v1, v2, cc in zip(c1_v, c2_v, counts)}
            res_0[(c1, c2)] = {(c1 + '_{}'.format(v1), c2 + '_{}'.format(v2)): cc for v1, v2, cc in zip(c1_v, c2_v, counts)}

    cols_distr = {}
    keys_pairs = list(set(list(res_0.keys()) + list(res_1.keys())))
    for c in cols:
        pairs = [k for k in keys_pairs if c in k]
        p_0 = []
        p_1 = []
        for kp in pairs:
            keys_count = list(set(list(res_0[kp].keys()) + list(res_1[kp].keys())))
            for kc in keys_count:
                p_0.append(res_0[kp].get(kc, 0))
                p_1.append(res_1[kp].get(kc, 0))
        cols_distr[c] = myjensenshannon(p_1, p_0, median_bool=True) * (len(cols) - 1) 

    print("==================================================")
    print(tipo_test)
    for k, v in {k: v for k, v in sorted(cols_distr.items(), key=lambda item: item[1])}.items():
        print(k, v)



TRAIN
rcat 0.07679046708780311
rnum 0.08388501629527509
embarked 0.08862449080965092
age 0.09334207995175585
fare 0.09334207995175585
pclass 0.09334207995175585
family_size 0.09707446160152443
is_alone 0.09707446160152443
title 0.16703279940553767
sex 0.21622539349926964
TEST
rcat 0.13466500884666732
rnum 0.13466500884666732
age 0.1362039097235961
family_size 0.13680242806944673
embarked 0.15181857769016405
is_alone 0.15181857769016405
pclass 0.16295914707815468
fare 0.17205951105608758
title 0.21695749468969427
sex 0.26327688477341593


**DISTANCIA JENSEN SHANNON (DJS)**

La distancia de Jensen-Shannon DJS satisface las siguientes propiedades:

$$
0 \leq DJS \leq \sqrt{\ln{2}} = 0.8325546111576977
$$


In [49]:
a1 = np.array([1, 0]) 
a0 = np.array([0, 1])
print(distance.jensenshannon(a1, a0))
print(distance.jensenshannon(a1, a0) / np.sqrt(2))

a1 = np.array([1, 0, 0])
a0 = np.array([0, 0, 1])
print(distance.jensenshannon(a1, a0) / np.sqrt(3))

a1 = np.array([1, 0, 0, 0])
a0 = np.array([0, 0, 0, 1])
print(distance.jensenshannon(a1, a0) / np.sqrt(4))

a1 = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0])
a0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1])
print(distance.jensenshannon(a1, a0) / np.sqrt(9))

0.8325546111576977
0.5887050112577372
0.48067562886696097
0.41627730557884884
0.2775182037192326


In [52]:
a1 = np.array([1, 0]) 
a0 = np.array([0, 1])
print(distance.jensenshannon(a1, a0))

a1 = np.array([1, 1, 0, 0]) 
a0 = np.array([0, 0, 1, 1])
print(distance.jensenshannon(a1, a0))

a1 = np.array([1, 1, 1, 1, 0, 0]) 
a0 = np.array([0, 0, 0, 0, 1, 1])
print(distance.jensenshannon(a1, a0))

0.8325546111576977
0.8325546111576977
0.8325546111576977


In [54]:
a1 = np.array([1, 1, 0]) 
a0 = np.array([0, 1, 1])
print(distance.jensenshannon(a1, a0))

a1 = np.array([1, 1, 1, 1, 0, 0]) 
a0 = np.array([0, 0, 1, 1, 1, 1])
print(distance.jensenshannon(a1, a0))

0.5887050112577373
0.5887050112577373


In [46]:
np.sqrt(np.log(2))

0.8325546111576977